# electricity_market_env

> This module provides a gymnasium based environment describing the electricity market dynamics.
this environment takes in mind the year seasons, day and night, production weather and price

In [ ]:
#| default_exp electricity_market_env

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import math
from enum import Enum
from io import BytesIO
from PIL import Image

import gymnasium as gym
import numpy as np
from gymnasium.core import RenderFrame
from gymnasium.envs.registration import register
from matplotlib import pyplot as plt
from sb3_contrib.common.wrappers import ActionMasker
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.monitor import Monitor

In [ ]:
#| export
class Season(Enum):
    FALL = 1
    WINTER = 2
    SPRING = 3
    SUMMER = 4

class Weather(Enum):
    SUNNY = 1
    CLOUDY = 2
    PARTIAL_CLOUDY = 3

WEATHER_PROBABILITIES_MAP_PER_SEASON = {
    Season.FALL: {
        Weather.SUNNY: 0.5,
        Weather.CLOUDY: 0.3,
        Weather.PARTIAL_CLOUDY: 0.2
    },
    Season.WINTER: {
        Weather.SUNNY: 0.2,
        Weather.CLOUDY: 0.5,
        Weather.PARTIAL_CLOUDY: 0.3
    },
    Season.SPRING: {
        Weather.SUNNY: 0.5,
        Weather.CLOUDY: 0.3,
        Weather.PARTIAL_CLOUDY: 0.2
    },
    Season.SUMMER: {
        Weather.SUNNY: 0.7,
        Weather.CLOUDY: 0.1,
        Weather.PARTIAL_CLOUDY: 0.2
    }
}

In [ ]:
#| export
class ElectricityMarketEnv(gym.Env):
    def __init__(self, env_config: dict | None = None, render_mode: str | None = None):
        if env_config is None:
            env_config = {}
        self._config = env_config
        self.render_mode = render_mode
        # each timestep is 4 hours
        self._timestep_duration_in_hours = 4
        self._episode_obs = []
        self._timestep = 0
        # Decided On granularity of 100 Wh
        if "battery_capacity" not in self._config:
            self._config["battery_capacity"] = 250  # default: 25 kWh
        self._battery_capacity = self._config["battery_capacity"]

        if "init_state_of_charge" not in self._config:
            self._config["init_state_of_charge"] = 200  # default: 20 kWh
        self._init_state_of_charge = self._config["init_state_of_charge"]
        self._current_state_of_charge = self._init_state_of_charge

        if "production_capacity" not in self._config:
            self._config["production_capacity"] = 720 / 6  # default: 72 kWh/day
        self._production_capacity = self._config["production_capacity"]

        if "max_timestep" not in self._config:
            self._config["max_timestep"] = 10_000
        self._max_timestep = self._config["max_timestep"]

        self._base_price = 0.065  # 0.65 NIS per kWh
        self._night_price_factor = 1.1
        self._high_demand_seasons_price_factor = 1.2
        self._battery_degradation = 0.9999

        self._night_demand_factor = 1.1
        self._high_demand_seasons_demand_factor = 1.2
        self._base_demand_of_electricity = 75  # 75 * 100 Wh per timestep

        # battery safe range, in percents
        self._battery_safe_range_percents = (20, 80)

        self.__weather = self._get_weather()
        self.__sell_price = self._get_sell_price()
        self.__demand_of_electricity = self._get_demand_of_electricity()
        self.action_space = gym.spaces.Discrete(2 * self._battery_capacity + 1)
        self.actions = list(range(-self._battery_capacity, self._battery_capacity + 1))
        self.observation_space = gym.spaces.Box(
            low=np.array([0, 0, 0, 0, 0, 0]),
            high=np.array([
                # Battery SoC
                self._current_state_of_charge / self._battery_capacity,
                # current battery capacity
                self._battery_capacity / self._battery_capacity,
                # battery safe range low boundary
                self._battery_safe_range[0] / self._battery_capacity,
                # battery safe range high boundary
                self._battery_safe_range[1] / self._battery_capacity,
                # Current electricity demand,
                self._max_demand_of_electricity / self._max_demand_of_electricity,
                # Current market price
                self._max_price / self._max_price
            ]),
            shape=(6,), dtype=np.float64
        )

    def _charge_amount(self, action: int) -> int:
        return math.ceil(self.actions[action] - self._demand_of_electricity)

    def _is_action_valid(self, action: int) -> bool:
        charge_amount = self._charge_amount(action)
        if charge_amount > self._production:
            return False
        target_state_of_charge = self._current_state_of_charge + charge_amount
        return 0 <= target_state_of_charge <= self._battery_capacity

    def step(self, action: int) -> tuple:
        charge_amount = self._charge_amount(action)
        done = self._battery_capacity == 0 or self._timestep >= self._max_timestep
        truncated = False
        if not self._is_action_valid(action):
            # print("Action {} is not valid".format(action))
            # truncated = True
            # observations = self._get_obs()
            reward = -1
            self._timestep += 1
            self.__weather = self._get_weather()
            self.__sell_price = self._get_sell_price()
            self.__demand_of_electricity = self._get_demand_of_electricity()
            observations = self._get_obs()
            self._episode_obs.append(observations)
            return observations, reward, done, truncated, {}

        self._current_state_of_charge += charge_amount
        self._battery_capacity *= self._battery_degradation
        # if violated the safe range, extra degradation
        if self._is_safe_range_violation:
            self._battery_capacity *= self._battery_degradation
        reward = self._reward(charge_amount)
        self._timestep += 1
        self.__weather = self._get_weather()
        self.__sell_price = self._get_sell_price()
        self.__demand_of_electricity = self._get_demand_of_electricity()
        observations = self._get_obs()
        self._episode_obs.append(observations)
        return observations, reward, done, truncated, {}

    def _get_weather(self) -> Weather:
        options, probs = zip(*WEATHER_PROBABILITIES_MAP_PER_SEASON[self._season].items())
        return np.random.choice(options, p=probs)

    @property
    def _weather(self) -> Weather:
        return self.__weather

    @property
    def _is_dark_hours(self) -> bool:
        # assuming night is 20:00-08:00 (dark hours)
        return self._timestep % 6 < 2 or self._timestep % 6 > 5

    @property
    def _season(self) -> Season:
        days_in_year = 365
        hours_in_day = 24
        number_of_seasons = 4
        timesteps_in_day = hours_in_day / self._timestep_duration_in_hours
        day_of_year = (self._timestep // timesteps_in_day) % days_in_year + 1
        days_in_season = math.ceil(days_in_year / number_of_seasons)
        match day_of_year // days_in_season:
            case 0:
                return Season.WINTER
            case 1:
                return Season.SPRING
            case 2:
                return Season.SUMMER
            case 3:
                return Season.FALL
            case _:
                raise ValueError(f"{days_in_season=} {day_of_year=}")

    @property
    def _production(self) -> float:
        # Solar panels doesn't produce at night.
        if self._is_dark_hours:
            return 0
        # Solar panels doesn't produce well on cloudy days
        match self._weather:
            case Weather.SUNNY:
                return self._production_capacity
            case Weather.CLOUDY:
                return self._production_capacity * 0.4
            case Weather.PARTIAL_CLOUDY:
                return self._production_capacity * 0.7
            case _:
                raise ValueError("Weather not supported")

    def _reward(self, charge_amount) -> float:
        max_reward = self._max_price * self._config["battery_capacity"]
        reward = abs(charge_amount) * self._sell_price
        # normalize (max reward is not reachable)
        normalized_reward = reward / max_reward
        if charge_amount >= 0:
            if self._is_safe_range_violation:
                return -normalized_reward
            return 0

        # penalty for violating the safe range
        if self._is_safe_range_violation:
            normalized_reward *= 0.3

        return normalized_reward

    @property
    def _demand_of_electricity(self) -> float:
        return self.__demand_of_electricity

    @property
    def _max_demand_of_electricity(self) -> float:
        return self._base_demand_of_electricity * self._high_demand_seasons_demand_factor * self._night_demand_factor * 1.2

    def _get_demand_of_electricity(self) -> float:
        demand = self._base_demand_of_electricity
        if self._season in (Season.SUMMER, Season.WINTER):
            demand *= self._high_demand_seasons_demand_factor
        if self._is_dark_hours:
            demand *= self._night_demand_factor
        noise = np.random.normal(-0.2 * demand, 0.2 * demand)
        return demand + noise

    @property
    def _is_safe_range_violation(self) -> bool:
        low, high = self._battery_safe_range
        return self._current_state_of_charge < low or self._current_state_of_charge > high

    @property
    def _battery_safe_range(self) -> tuple[float, float]:
        low, high = self._battery_safe_range_percents
        return low * self._battery_capacity / 100, high * self._battery_capacity / 100

    @property
    def _max_price(self) -> float:
        # base price in dark hours in winter/summer with max noise
        return self._base_price * self._night_price_factor * self._high_demand_seasons_price_factor * 1.2

    def _get_sell_price(self) -> float:
        price = self._base_price
        if self._is_dark_hours:
            price *= self._night_price_factor
        if self._season in (Season.WINTER, Season.SUMMER):
            price *= self._high_demand_seasons_price_factor
        noise = np.random.normal(-price * 0.2, price * 0.2)
        return price + noise

    @property
    def _sell_price(self) -> float:
        return self.__sell_price

    def reset(self, *, seed: int | None = None, options: dict | None = None):
        """ Resets the environment to the initial state. """
        super().reset(seed=seed, options=options)
        observations = self._get_obs()
        self._episode_obs = [observations]
        self._timestep = 0
        self._current_state_of_charge = self._init_state_of_charge
        self._battery_capacity = self._config["battery_capacity"]
        self.__weather = self._get_weather()
        self.__sell_price = self._get_sell_price()
        self.__demand_of_electricity = self._get_demand_of_electricity()
        return observations, {}

    def action_masks(self) -> np.ndarray:
        """Generate a boolean mask of valid actions for `MaskablePPO`."""
        mask = np.array(
            [self._is_action_valid(action) for action in range(self.action_space.n)], dtype=bool
        )
        if not np.any(mask):  # If all actions are invalid, force one to be valid
            mask[len(mask) // 2] = True
        return mask

    def _get_obs(self) -> np.ndarray:
        """ Returns the current observation (state). """
        return np.array([
            # Battery SoC
            self._current_state_of_charge / self._battery_capacity,
            # current battery capacity
            self._battery_capacity / self._battery_capacity,
            # battery safe range low boundary
            self._battery_safe_range[0] / self._battery_capacity,
            # battery safe range high boundary
            self._battery_safe_range[1] / self._battery_capacity,
            # Current electricity demand
            self._demand_of_electricity / self._max_demand_of_electricity,
            # Current market price
            self._sell_price / self._max_price
        ], dtype=np.float64)

    def render(self) -> RenderFrame | list[RenderFrame] | None:
        if not self._episode_obs or self._timestep < self._max_timestep:
            return []

        observations = np.array(self._episode_obs)
        timesteps = np.arange(len(observations))
        feature_labels = ["Battery Level", "Battery Capacity", "Electricity Demand"]

        # Create a figure with two subplots: one for Battery Level & Electricity Demand, and one for Price
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

        # Plot Battery Level, Battery Capacity, and Electricity Demand in the first subplot (ax1)
        ax1.plot(timesteps, observations[:, 0], label=feature_labels[0])  # Battery Level (SoC)
        ax1.plot(timesteps, observations[:, 1], label=feature_labels[1])  # Battery Capacity
        ax1.plot(timesteps, observations[:, 4], label="Electricity Demand")  # Electricity Demand

        # Use the safe range boundaries from observations (indices 2 and 3)
        low = observations[:, 2]  # Safe range low (from observation)
        high = observations[:, 3]  # Safe range high (from observation)

        # Plot the safe range boundaries (red lines)
        ax1.plot(timesteps, low, color='red', linestyle='--', label="Battery Safe Range Low")
        ax1.plot(timesteps, high, color='red', linestyle='--', label="Battery Safe Range High")

        # Add labels, title, and legend for the first plot
        ax1.set_xlabel("Timestep")
        ax1.set_ylabel("Value")
        ax1.set_title("Battery Level, Battery Capacity & Electricity Demand Over Time")
        ax1.legend()

        # Plot the Price in the second subplot (ax2)
        ax2.plot(timesteps, observations[:, 5], label="Price", color="orange")  # Price from observation

        # Optionally, customize the second plot scale (e.g., different y-limits)
        ax2.set_ylim(bottom=0)  # Adjust this as needed for your price range

        # Add labels, title, and legend for the second plot
        ax2.set_xlabel("Timestep")
        ax2.set_ylabel("Price (in NIS/kWh)")
        ax2.set_title("Electricity Price Over Time")
        ax2.legend()

        plt.show()

        # Convert plot to an image
        buf = BytesIO()
        plt.savefig(buf, format="png")
        plt.close()
        buf.seek(0)

        img = Image.open(buf)
        return [img]


In [ ]:
#| hide
env = ElectricityMarketEnv({"max_timestep": 10_000})
check_env(env)

In [ ]:
#| hide
def mask_fn(env: ElectricityMarketEnv) -> np.ndarray:
    return env.action_masks()

env = ActionMasker(ElectricityMarketEnv({"max_timestep": 10_000},
                                        render_mode="human"), mask_fn)
check_env(env)

In [ ]:
#| hide
env = Monitor(ActionMasker(ElectricityMarketEnv({"max_timestep": 10_000}, render_mode="human"), mask_fn))
check_env(env)

In [ ]:
#| hide
register(
    id='ElectricityMarketEnv-v0',
    entry_point='electricity_market.electricity_market_env:ElectricityMarketEnv',
)

/Users/ofer/Downloads/02360018_final_project/.venv/lib/python3.12/site-packages/gymnasium/envs/registration.py:642: UserWarning: WARN: Overriding environment ElectricityMarketEnv-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()